In [1]:
from datetime import datetime
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
import numpy as np
from binance.client import Client
from tqdm import tqdm  # 用于进度条显示

In [2]:
api_key = 'bNpXIJwWOwPitmuUoKZsyuPV8lI1mcJV4iljO5gUrV5oShZzT98qaYxn7vVyIlLJ'
api_secret = '5oO1t4GwVq9Q3ixm2aequZije30NkyA2Pv6ZQAIB2za8TovVXbWWS3HxD9uBv2ZX'

In [3]:
client = Client(api_key, api_secret)

In [4]:

def fetch_1m_data(symbol, start_date='2023-05-05 16:00:00', end_date='2024-12-03 00:00:00'):
    """
    获取指定交易对从给定开始日期以来的1分钟级数据（K线数据）。
    """
    start_time = int(datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S").timestamp() * 1000)
    end_time = int(datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S").timestamp() * 1000)
    klines = client.get_historical_klines(symbol, '1m', start_time, end_time)

    data = []
    for k in klines:
        data.append({
            'open_time': pd.to_datetime(k[0], unit='ms'),
            'open': float(k[1]),
            'high': float(k[2]),
            'low': float(k[3]),
            'close': float(k[4]),
            'volume': float(k[5]),
            'close_time': pd.to_datetime(k[6], unit='ms'),
            'quote_asset_volume': float(k[7]),
            'number_of_trades': int(k[8]),
            'taker_buy_base_asset_volume': float(k[9]),
            'taker_buy_quote_asset_volume': float(k[10])
        })

    df = pd.DataFrame(data)
    return df

def process_and_save_data(symbols, 
                          start_date='2015-01-01 00:00:00',
                          end_date=None,
                          raw_file_suffix='_1m_raw.csv',
                          ta_file_suffix='_1m_ta.csv'):
    """
    对给定的币对列表一次性获取1分钟数据，并进行指标计算后保存，并加入进度条显示处理进度。
    """
    if end_date is None:
        end_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # 定义自定义策略
    my_strategy = ta.Strategy(
        name="My Custom 50 Indicators Strategy",
        ta=[
            {"kind": "rsi", "length": 14},
            {"kind": "rsi", "length": 7},
            {"kind": "stoch", "k":14, "d":3, "smooth":3},
            {"kind": "stochrsi", "length":14},
            {"kind": "macd", "fast":12, "slow":26, "signal":9},
            {"kind": "ppo", "fast":12, "slow":26, "signal":9},
            {"kind": "pvo", "fast":12, "slow":26, "signal":9},
            {"kind": "ao"},
            {"kind": "mfi", "length":14},
            {"kind": "cmf", "length":20},
            {"kind": "obv"},
            {"kind": "ad"},
            {"kind": "eom", "length":14},
            {"kind": "roc", "length":10},
            {"kind": "mom", "length":10},
            {"kind": "cci", "length":20},
            {"kind": "willr", "length":14},
            {"kind": "aroon", "length":14},
            {"kind": "dpo", "length":20},
            {"kind": "chop", "length":14},
            {"kind": "adx", "length":14},
            {"kind": "trix", "length":14},
            {"kind": "kst", "roc1":10, "roc2":15, "roc3":20, "roc4":30, "signal":9},
            {"kind": "tsi", "long":25, "short":13},
            {"kind": "fisher", "length":9},
            {"kind": "uo", "fast":7, "medium":14, "slow":28},
            {"kind": "rvi", "length":14},
            {"kind": "atr", "length":14},
            {"kind": "bbands", "length":20, "std":2},
            {"kind": "kc", "length":20},
            {"kind": "donchian", "lower_length":20, "upper_length":20},
            {"kind": "psar"},
            {"kind": "supertrend", "length":10, "multiplier":3.0},
            {"kind": "ichimoku"},  # 会生成多个列
            {"kind": "linreg", "length":14},
            {"kind": "ema", "length":200},
            {"kind": "ema", "length":50},
            {"kind": "ema", "length":21},
            {"kind": "sma", "length":200},
            {"kind": "sma", "length":50},
            {"kind": "sma", "length":21},
            {"kind": "dema", "length":14},
            {"kind": "tema", "length":14},
            {"kind": "wma", "length":14},
            {"kind": "hma", "length":14},
            {"kind": "t3", "length":14},
        ]
    )
    
    # 使用tqdm添加进度条
    for symbol in tqdm(symbols, desc="Processing symbols"):
        df_1m = fetch_1m_data(symbol, start_date, end_date)
        df_1m = df_1m.drop(['close_time', 'quote_asset_volume', 'number_of_trades', 
                            'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume'], axis=1)
        df_1m['date'] = df_1m['open_time']
        df_1m = df_1m.drop(['open_time'], axis=1)
        df_1m = df_1m.set_index('date')
        df_1m['return'] = df_1m['close'].pct_change()
        
        # 保存原始数据
        raw_filename = symbol + raw_file_suffix
        df_1m.to_csv(raw_filename)
        
        # 使用自定义策略对数据进行技术指标计算
        df_1m.ta.strategy(my_strategy)
        
        # 保存包含技术指标的数据
        ta_filename = symbol + ta_file_suffix
        df_1m.to_csv(ta_filename)

    print("All symbols processed.")

In [5]:
symbols = [ "NEOUSDT", "PENDLEUSDT",
    "ONGUSDT", "SAGAUSDT", "BELUSDT", "ONTUSDT", "HBARUSDT", "WUSDT", "GLMUSDT", "NOTUSDT",
    "BBUSDT", "HIGHUSDT", "EDUUSDT", "IOUSDT", "CRVUSDT", "TONUSDT", "1000RATSUSDT", "LISTAUSDT",
    "LDOUSDT", "ZKUSDT", "ZROUSDT", "BNXUSDT", "TURBOUSDT", "BONDUSDT", "MEWUSDT", "STMXUSDT",
    "AAVEUSDT", "RAREUSDT", "TRXUSDT", "SUNUSDT", "DOGSUSDT", "VIDTUSDT", "REEFUSDT", "NEIROETHUSDT",
    "POPCATUSDT", "BIGTIMEUSDT", "NEIROUSDT", "UXLINKUSDT", "TAOUSDT", "CATIUSDT", "1MBABYDOGEUSDT",
    "EIGENUSDT", "HMSTRUSDT", "APEUSDT", "DIAUSDT"
]

In [6]:
# 获取 Binance 支持的所有交易对
exchange_info = client.get_exchange_info()
valid_symbols = [s['symbol'] for s in exchange_info['symbols']]

# 找出无效的交易对
invalid_symbols = [symbol for symbol in symbols if symbol not in valid_symbols]
print("无效的交易对:", invalid_symbols)

# 删除无效交易对
symbols = [symbol for symbol in symbols if symbol in valid_symbols]

# 输出修正后的交易对列表
print("有效的交易对:", symbols)

无效的交易对: ['ONDOUSDT', '1000RATSUSDT', 'MEWUSDT', 'NEIROETHUSDT', 'POPCATUSDT', 'BIGTIMEUSDT', 'UXLINKUSDT']
有效的交易对: ['FTMUSDT', 'BOMEUSDT', 'POLYXUSDT', 'ETHFIUSDT', 'FRONTUSDT', 'ENAUSDT', 'YGGUSDT', 'AEVOUSDT', 'NEOUSDT', 'PENDLEUSDT', 'ONGUSDT', 'SAGAUSDT', 'BELUSDT', 'ONTUSDT', 'HBARUSDT', 'WUSDT', 'GLMUSDT', 'NOTUSDT', 'BBUSDT', 'HIGHUSDT', 'EDUUSDT', 'IOUSDT', 'CRVUSDT', 'TONUSDT', 'LISTAUSDT', 'LDOUSDT', 'ZKUSDT', 'ZROUSDT', 'BNXUSDT', 'TURBOUSDT', 'BONDUSDT', 'STMXUSDT', 'AAVEUSDT', 'RAREUSDT', 'TRXUSDT', 'SUNUSDT', 'DOGSUSDT', 'VIDTUSDT', 'REEFUSDT', 'NEIROUSDT', 'TAOUSDT', 'CATIUSDT', '1MBABYDOGEUSDT', 'EIGENUSDT', 'HMSTRUSDT', 'APEUSDT', 'DIAUSDT']


In [7]:


# 假设 process_and_save_data 函数已经定义，并且 client 已正确初始化。
process_and_save_data(
    symbols, 
    start_date='2021-01-01 00:00:00', 
    end_date=None,  # 使用当前时间
    raw_file_suffix='_1m_raw.csv',
    ta_file_suffix='_1m_ta.csv'
)

Processing symbols:  19%|█▉        | 9/47 [2:05:04<8:48:04, 833.81s/it]


ProxyError: HTTPSConnectionPool(host='api.binance.com', port=443): Max retries exceeded with url: /api/v3/klines?endTime=1735271215471&interval=1m&limit=1&startTime=0&symbol=PENDLEUSDT (Caused by ProxyError('Cannot connect to proxy.', ConnectionAbortedError(10053, '你的主机中的软件中止了一个已建立的连接。', None, 10053, None)))